# 📊 Social Media Sentiment Analysis
This notebook analyzes Twitter data to understand public sentiment towards a specific topic using NLP and Sentiment Analysis.
### Internship Task - Brainwave Matrix Solutions
---

In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import re
import nltk
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

nltk.download('stopwords')

## 1. Collect Tweets

In [ ]:
query = "Data Analytics"   # 🔹 change topic here
limit = 200  # number of tweets

tweets = []
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    tweets.append([tweet.date, tweet.user.username, tweet.content])

df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
df.head()

## 2. Clean Tweets

In [ ]:
stop = set(stopwords.words('english'))

def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet)
    tweet = re.sub(r'\@\w+|\#','', tweet)
    tweet = tweet.lower()
    tweet = " ".join([word for word in tweet.split() if word not in stop])
    return tweet

df['Clean_Tweet'] = df['Tweet'].apply(clean_tweet)
df.head()

## 3. Sentiment Analysis

In [ ]:
analyzer = SentimentIntensityAnalyzer()

df['Sentiment'] = df['Clean_Tweet'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

def get_sentiment(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df['Sentiment_Label'] = df['Sentiment'].apply(get_sentiment)
df.head()

## 4. Visualization

In [ ]:
# Pie Chart
df['Sentiment_Label'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['green','red','grey'])
plt.title("Sentiment Distribution")
plt.show()

# Trend over time
df.groupby(df['Date'].dt.date)['Sentiment'].mean().plot(kind='line')
plt.title("Sentiment Trend Over Time")
plt.ylabel("Average Sentiment Score")
plt.show()

## 5. Export Results

In [ ]:
df.to_excel("Sentiment_Analysis_Report.xlsx", index=False)
print("Report saved as Sentiment_Analysis_Report.xlsx")